In [ ]:
# If running from Colab, please uncomment and run the following cell to get summer installed.
#!pip install summerepi2==1.3.5

In [ ]:
import pandas as pd
from datetime import datetime
from plotly import graph_objects as go
from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter, DerivedOutput
pd.options.plotting.backend = 'plotly'

## Epidemiological explanation
Here we take a similar epidemiological model to that presented in 
the `seir_model.ipynb` notebook and apply a simple stratification to it.
### Age stratification
The stratification itself is named 'age', but does not incorporate demographic processes
(because we only use the standard 'Stratification' object).
The stratification has no epidemiological effect on the model behaviour,
execept that it permits us to generate some age group-specific outputs
(and in the second model apply some different flow rates).
### Tracking death rates
The purpose of this stratification is to allow calculation of death rates,
with the infection fatality rate (IFR) being allowed to differ according by age group.
In the first model, we track recoveries and multiply the rate of recovery for each stratum 
by an age-specific IFR to calculate the total rate of deaths for the population.
This means that we have a "closed population" and so just track deaths
as a derived process linked to incidence.
### Parameters
Some additional parameters are needed here, 
including the distribution of population by age group.
None of the parameters are intended to be exact,
but are presented to allow the user to adjust them to desired (and more realistic) ranges.

In [ ]:
iso = 'MYS'

In [ ]:
deaths_data = pd.read_csv('https://github.com/monash-emu/wpro_working/raw/main/data/new_deaths.csv', index_col=0)[iso]
deaths_data.index = pd.to_datetime(deaths_data.index)
approx_pops = {
    'MYS': 33e6,
    'PHL': 114e6,
    'VNM': 97e6,
}
analysis_start_date = datetime(2021, 8, 1) 
analysis_end_date = datetime(2022, 6, 1)

In [ ]:
def get_epi_model():
    epi_model = CompartmentalModel(
        [analysis_start_date, analysis_end_date],
        ['susceptible', 'exposed', 'infectious', 'recovered'],
        ['infectious'],
        ref_date=datetime(2019, 12, 31),
    )
    epi_model.add_infection_frequency_flow('infection', Parameter('contact_rate'), 'susceptible', 'exposed')
    epi_model.add_transition_flow('progression', 1.0 / Parameter('incubation_period'), 'exposed', 'infectious')
    epi_model.add_transition_flow('recovery', 1.0 / Parameter('infectious_period'), 'infectious', 'recovered')
    epi_model.set_initial_population({'susceptible': approx_pops[iso], 'infectious': 1.0})
    return epi_model

age_strata = ['0', '15', '60', '75']
age_pop_split = {'0': 0.2, '15': 0.5, '60': 0.2, '75': 0.1}

def get_age_strat(comp_names):
    age_strat = Stratification('age', age_strata, comp_names)
    age_strat.set_population_split(age_pop_split)
    return age_strat

plot_start_date = datetime(2021, 12, 1)

def plot_age_deaths(model):
    fig = model.get_derived_outputs_df().plot.area()
    fig.add_trace(go.Scatter(x=deaths_data.index, y=deaths_data, name="reported"))
    fig.update_layout(yaxis_range=[0, 150], xaxis_range=[plot_start_date, analysis_end_date])
    return fig

In [ ]:
# Get model and stratify by age
epi_model_1 = get_epi_model()
age_strat = get_age_strat(epi_model_1._original_compartment_names)
epi_model_1.stratify_with(age_strat)

# Calculate deaths as a proportion of the recovery flow
for age in age_strata:
    age_recoveries = epi_model_1.request_output_for_flow(f'rec_{age}', 'recovery', source_strata={'age': age}, save_results=False)
    epi_model_1.request_function_output(f'deaths_{age}', age_recoveries * Parameter(f'ifr_{age}'))

In [ ]:
# Specify parameters, run and plot outputs
parameters = {
    'contact_rate': 0.4,
    'incubation_period': 5.0,
    'infectious_period': 5.0,
    'ifr_0': 0.0, 
    'ifr_15': 0.0, 
    'ifr_60': 0.0002, 
    'ifr_75': 0.002,
}
epi_model_1.run(parameters)
plot_age_deaths(epi_model_1)

### Using an explicit death flow
In this alternative version of the model, 
we will apply a death flow using `summer2`'s 
death flow method, which means that people actually leave 
the model as they die (and so the model is no longer "closed").
We'll run it with the same parameters and should expect similar
(although not quite identical) dynamics.

In [ ]:
# Get model and add an explicit deaths flow
epi_model_2 = get_epi_model()
epi_model_2.add_death_flow('death', 1.0 / Parameter('infectious_period'), 'infectious')

# Modify deaths and recovery flows according to IFRs
age_strat_2 = get_age_strat(epi_model_2._original_compartment_names)
age_strat_2.set_flow_adjustments('death', {k: Parameter(f'ifr_{k}') for k in age_strata})
age_strat_2.set_flow_adjustments('recovery', {k: 1.0 - Parameter(f'ifr_{k}') for k in age_strata})
epi_model_2.stratify_with(age_strat_2)
for age in age_strata:
    epi_model_2.request_output_for_flow(f'deaths_{age}', 'death', source_strata={'age': age})

In [ ]:
# Specify parameters, run and plot outputs
epi_model_2.run(parameters)
plot_age_deaths(epi_model_2)

In [ ]:
# Compare the total population under either approach
pd.concat([epi_model_1.get_outputs_df().sum(axis=1), epi_model_2.get_outputs_df().sum(axis=1)], axis=1).plot()